# HW 3-Part-of-Speech Tagging with HMMs + Decoding Techniques (Greedy and Viterbi)

- Detravious Jamari Brinkley
- CSCI-544: Applied Natural Language Processing
- python version: 3.11.4

---

1. Part-of-Speech (POS) Tagging [a type of sequence labelling task where of a given word, assign the part of speech]
2. HMMs (Hidden Markov Model) [a generative-based model that's used for POS Tagging]
    1. Generative-based [provides the probabilities for all possible combinations of values of variables in the set using the joint distribution]
    2. With POS Tagging: Given a sequence of observations (sentences), the task is to infer the most likely sequence of hidden states (POS Tags) that could have generated the observed data.
3. **Decoding Techniques:**
    1. Greedy [find the optimal (OPT) solution at each step]
    2. Viterbi [make use of dynammic programming to find the OPT solution with backtracking while searching the entire search space]
4. **Notes of the data and given files:**
    - Dataset: Wall Street Journal section of the Penn Treebank
    - Folder named `data` with the following files:
        1. `train`, sentences *with* human-annotated POS Tags
        2. `dev`, sentences *with* human-annotated POS Tags
        3. `test`, sentences *without* POS Tags, thus predict the POS Tags
    - Format: Blank like at the end of each sentence. Each line contains 3 items separated by the `\t`, the tab symbol. These three items are
        1. Index of the word in the sentence
        2. Word type
        3. POS Tag



In [1]:
import sys
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict

# Load and Update Data
- [x] Find a way to separate sentences when loading the df.

In [2]:
def load_data(file_path: str, file_name: str, is_test_file: bool, config_index: bool = True):
    
    if config_index == True:
        if is_test_file != True:
            file =  file_path + file_name
            open_df = pd.read_table(file, sep = "\t", names=['Index', 'Word', 'POS Tag'], skip_blank_lines=False)
        else:
            file =  file_path + file_name
            open_df = pd.read_table(file, sep = "\t", names=['Index', 'Word'], skip_blank_lines=False)
        
    return open_df

In [3]:
def update_df_rows_with_dummy(df: pd.DataFrame, new_columns_name: list) -> pd.DataFrame:  
    """Update the rows of the dataframe if blank space, fill with dummy"""  

    dummy_row = pd.DataFrame([['0.0', ' ', 'dummy']], columns=df.columns)
    df = pd.concat([dummy_row, df], ignore_index=True)
    df.columns = new_columns_name
    df.fillna("dummy", inplace=True)
   
    return df

In [4]:
train_df = load_data('data/', 'train', False)
dev_df = load_data('data/', 'dev', False)
test_df = load_data('data/', 'test', True)

In [5]:
train_dev_columns_name = ['Index', 'Word', 'POS Tag']

updated_train_df = update_df_rows_with_dummy(train_df, train_dev_columns_name)
updated_dev_df = update_df_rows_with_dummy(dev_df, train_dev_columns_name)

In [6]:
all_pos_tags = updated_train_df['POS Tag'].unique()
all_pos_tags

array(['dummy', 'NNP', ',', 'CD', 'NNS', 'JJ', 'MD', 'VB', 'DT', 'NN',
       'IN', '.', 'VBZ', 'VBG', 'CC', 'VBD', 'VBN', 'RB', 'TO', 'PRP',
       'RBR', 'WDT', 'VBP', 'RP', 'PRP$', 'JJS', 'POS', '``', 'EX', "''",
       'WP', ':', 'JJR', 'WRB', '$', 'NNPS', 'WP$', '-LRB-', '-RRB-',
       'PDT', 'RBS', 'FW', 'UH', 'SYM', 'LS', '#'], dtype=object)

# Outline of Tasks

1. Vocabulary Creation
2. Model Learning
3. Greedy Decoding with HMM
4. Viterbi Decoding with HMM


# 1. Vocabulary Creation

- **Problem:** Creating vocabulary to handle unkown words.
    - **Solution:** Replace rare words wtih whose occurrences are less than a threshold (ie: 3) with a special token `< unk >`

---

1. [x] Create a vocabulary using the training data in the file train
2. [x] Output the vocabulary into a txt file named `vocab.txt`
    - [x] See PDF on how to properly format vocabulary file
3. [ ] Questions
    1. [ ] What is the selected threshold for unknown words replacement?
    2. [ ] What is the total size of your vocabulary?
    3. [ ] What is the total occurrences of the special token `< unk >`after replacement?

In [7]:
true_false_series = updated_train_df['Word'].value_counts()
vocab_df = pd.DataFrame(true_false_series)
vocab_df.reset_index(inplace = True)
vocab_df

,Word,count
0,",",46476
1,the,39533
2,dummy,38234
3,.,37452
4,of,22104
...,...,...
43188,Birthday,1
43189,Happy,1
43190,Bertie,1
43191,crouched,1


In [8]:
def create_vocab_threshold_df(df: pd.DataFrame, word_col_name: str, count_col_name: str, threhold: int, special_token: str, save_df: bool, save_path_with_name: str):
    """For every word in df, replace with special_token if below threshold
    
    """
    true_false_series = df[count_col_name] > 3
    
    updated_vocab_df = df.loc[true_false_series == True]
    updated_false_vocab_df = df.loc[true_false_series == False]
    updated_false_vocab_df[word_col_name] = special_token
    
    N_updated_false_vocab_df = len(updated_false_vocab_df)
    
    new_row = pd.DataFrame([[special_token, N_updated_false_vocab_df]], columns=updated_vocab_df.columns)
    final_df = pd.concat([new_row, updated_vocab_df], ignore_index=True)
    N_vocab = range(0, len(updated_vocab_df)+1)
    
    final_df["index"] = N_vocab
    
    final_df = final_df.reindex(columns=[word_col_name, "index", count_col_name])
    if save_df == True:
        final_df.to_csv(save_path_with_name, header=None, index=None, sep='\t')
    
    return final_df

In [9]:
word_col_name = "Word"
count_col_name = "count"
special_token = "< unk >"
save_df = False
save_file_path_and_name = "submit/vocab.txt"
updated_vocab_df = create_vocab_threshold_df(vocab_df, word_col_name, count_col_name, 3, special_token, save_df, save_file_path_and_name)

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_47409/3585010688.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_false_vocab_df[word_col_name] = special_token


In [10]:
updated_vocab_df

,Word,index,count
0,< unk >,0,29443
1,",",1,46476
2,the,2,39533
3,dummy,3,38234
4,.,4,37452
...,...,...,...
13746,trafficking,13746,4
13747,7.62,13747,4
13748,gut,13748,4
13749,17.3,13749,4


# 2. Model Learning

- Learn an HMM from the training data
- **HMM Parameters:**
  <div style="text-align: center;">

    $
    \text{Transition Probability (} t \text{)}: \quad t(s' \mid s) = \frac{\text{count}(s \rightarrow s')}{\text{count}(s)}
    $

    $
    \text{Emission Probability (} e \text{)}: \quad e(x \mid s) = \frac{\text{count}(s \rightarrow x)}{\text{count}(s)}
    $

  </div>

---

1. [x] Learn a model using the training data in the file train
2. [x] Output the learned model into a model file in json format, named `hmm.json`. The model file should contains two dictionaries for the emission and transition parameters, respectively.
    1. [x] 1st dictionary: Named transition, contains items with pairs of (s, s′) as key and t(s′|s) as value. 
    2. [x] 2nd dictionary: Named emission, contains items with pairs of (s, x) as key and e(x|s) as value.
3. Question
    1. [ ] How many transition and emission parameters in your HMM?


In [11]:
# updated_train_df.head(20)

In [12]:
def hmm(df, word_col_name, pos_tag_col_name, prev_pos_tag_col_name):
    transition_states = defaultdict(int)
    emission_state_word = defaultdict(int)
    N_state = defaultdict(int)
    
    df[prev_pos_tag_col_name] = df[pos_tag_col_name].shift(1) # previous state for trnasition probabilities

    # iterate through vocabulary
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):

        emission_state_word[(row[pos_tag_col_name], row[word_col_name])] += 1
        # transition count + 1
        if pd.notnull(row[prev_pos_tag_col_name]):  # Check if it's not NaN
            transition_states[(row[prev_pos_tag_col_name], row[pos_tag_col_name])] += 1

        # increment tag when I see it
        N_state[(row[pos_tag_col_name])] += 1

    return emission_state_word, transition_states, N_state

In [13]:
pos_tag_col_name = "POS Tag"
prev_pos_tag_col_name = 'Previous_POS Tag'
emissions, transitions, N_states = hmm(updated_train_df, word_col_name, pos_tag_col_name, prev_pos_tag_col_name)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 950313/950313 [00:47<00:00, 19811.68it/s]


In [14]:
def calculate_prob(transitions: dict, emissions: dict, N_states: dict, prob_type: str):   

    if prob_type == "t":
        t_or_e = transitions
    elif prob_type == "e":
        t_or_e = emissions

    store_probs = {}
    for key, value in t_or_e.items():
        
        curr_state = key[0]
        # print(key)
       
        store_probs[key] = value / N_states[curr_state]
        
    return store_probs

In [15]:
t_probs = calculate_prob(transitions, emissions, N_states, 't')
e_probs = calculate_prob(transitions, emissions, N_states, 'e')

In [16]:
# list(t_probs.items())[:7]

In [17]:
list(e_probs.items())[:7]

[(('dummy', ' '), 2.6165681092678842e-05),
 (('NNP', 'Pierre'), 6.84868961738654e-05),
 (('NNP', 'Vinken'), 2.2828965391288468e-05),
 ((',', ','), 0.9999139414802065),
 (('CD', '61'), 0.0007168253240050465),
 (('NNS', 'years'), 0.019530237301024905),
 (('JJ', 'old'), 0.003613599348534202)]

In [18]:
save_path_with_name = "submit/hmm.json"

combine_t_and_e_probs = {}
combine_t_and_e_probs["transitions"] = t_probs
combine_t_and_e_probs["emissions"] = e_probs

t_e_probs_df = pd.DataFrame(combine_t_and_e_probs)
# t_e_probs_df.to_json(save_path_with_name)

# json_object = json.dumps(combine_t_and_e_probs)
# with open(save_path_with_name, 'w') as json_file:
#     json_file.write(json_object)

# 3. Greedy Decoding with HMM

1. [ ] Implement the greedy decoding algorithm
2. [ ] Evaluate it on the development data
3. [ ] Predicting the POS Tags of the sentences in the test data
4. [ ] Output the predictions in a file named `greedy.out`, in the same format of training data
5. [ ] Evaluate the results of the model on `eval.py` in the terminal with `python eval.py − p {predicted file} − g {gold-standard file}`
6. [ ] Question
    1. [ ] What is the accuracy on the dev data? 

In [19]:
# updated_dev_df.head(40)

In [63]:
def greedy_decoding(dev_df: pd.DataFrame, t_probs: dict, e_probs: dict, N_pos_tags: np.array):
    """Implement greedy decoding on the development file (words only) using the transition probability and emission probability. 
    Furthermore, don't use POS Tag of development file, thus only use POS Tag from training data.

    Parameters
    ----------
    df: `pd.DataFrame`
        Dev file

    t_probs: `py dict`
        Tranision probabilities for POS Tag given previous POS Tag

    e_probs: `py dict`
        Emission probabilities for Word given POS Tags

    N_pos_tags: `np.array`
        All POS Tags found in the training file
    
    Return
    ------
    
    """

    previous_pos_tag = "dummy"
    all_words_with_pos_tag = {}
    
    for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        print("index", index, "with word", row['Word'])
        
        store_scores = []
        for N_pos_tags_idx in range(len(N_pos_tags)):
            current_pos_tag = N_pos_tags[N_pos_tags_idx]
            print("- Current POS Tag: ", current_pos_tag)

            """Transition"""
            t_find_pos_tag = current_pos_tag
            t_given_pos_tag = previous_pos_tag
            # print(f"--- t({t_find_pos_tag} | {t_given_pos_tag})")
            
            """Emission"""
            e_word = row['Word']
            e_given_pos_tag = current_pos_tag
            # print(f"--- e({e_given_pos_tag} | {e_word})") # order this way to match e_probs dictionary
            
            """Transition * Emission"""
            t_key = (t_find_pos_tag, t_given_pos_tag)
            e_key = (e_given_pos_tag, e_word)
            # print(t_key in t_probs, e_key in e_probs)

            # IF-ELSE bc not all pairs will be found. If pair is found, use score, otherwise (pair isn't found) set score to 0.0.
            if t_key in t_probs and e_key in e_probs:
                t = t_probs[t_key]
                e = e_probs[e_key]
                score = t * e
                print(f"---  t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                
            else:
                t = 0.0
                e = 0.0
                score = t * e
                print(f"--- t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                        
            store_scores.append(score)
        print(f"Scores", store_scores)
    
            # print("Word is: ", row['Word'], "with POS Tag of", current_pos_tag)
        max_score_idx = np.argmax(np.array(store_scores)) # use argmax to get the index of max score
        
        current_pos_tag = N_pos_tags[max_score_idx] # use the index of the max score to find which POS Tag to update to
        print("Word is: ", row['Word'], "with POS Tag of", current_pos_tag)
        all_words_with_pos_tag[row['Word']] = current_pos_tag
        print(all_words_with_pos_tag)

        # print("Updated POS Tag", current_pos_tag)
        print()
        
    
    return all_words_with_pos_tag

In [65]:
gd_output = greedy_decoding(updated_dev_df[:30], t_probs, e_probs, all_pos_tags)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 1583.39it/s]

index 0 with word  
- Current POS Tag:  dummy
--- t(dummy | dummy) * e(  | dummy) = 0.0
- Current POS Tag:  NNP
--- t(NNP | dummy) * e(  | NNP) = 0.0
- Current POS Tag:  ,
--- t(, | dummy) * e(  | ,) = 0.0
- Current POS Tag:  CD
--- t(CD | dummy) * e(  | CD) = 0.0
- Current POS Tag:  NNS
--- t(NNS | dummy) * e(  | NNS) = 0.0
- Current POS Tag:  JJ
--- t(JJ | dummy) * e(  | JJ) = 0.0
- Current POS Tag:  MD
--- t(MD | dummy) * e(  | MD) = 0.0
- Current POS Tag:  VB
--- t(VB | dummy) * e(  | VB) = 0.0
- Current POS Tag:  DT
--- t(DT | dummy) * e(  | DT) = 0.0
- Current POS Tag:  NN
--- t(NN | dummy) * e(  | NN) = 0.0
- Current POS Tag:  IN
--- t(IN | dummy) * e(  | IN) = 0.0
- Current POS Tag:  .
--- t(. | dummy) * e(  | .) = 0.0
- Current POS Tag:  VBZ
--- t(VBZ | dummy) * e(  | VBZ) = 0.0
- Current POS Tag:  VBG
--- t(VBG | dummy) * e(  | VBG) = 0.0
- Current POS Tag:  CC
--- t(CC | dummy) * e(  | CC) = 0.0
- Current POS Tag:  VBD
--- t(VBD | dummy) * e(  | VBD) = 0.0
- Current POS Tag:

In [23]:
gd_output

{' ': 'dummy',
 'The': 'DT',
 'Arizona': 'NNP',
 'Corporations': 'NNS',
 'Commission': 'NNP',
 'authorized': 'VBD',
 'an': 'DT',
 '11.5': 'CD',
 '%': 'NN',
 'rate': 'NN',
 'increase': 'NN',
 'at': 'IN',
 'Tucson': 'NNP',
 'Electric': 'NNP',
 'Power': 'NNP',
 'Co.': 'NNP',
 ',': ',',
 'substantially': 'RB',
 'lower': 'JJR',
 'than': 'IN',
 'recommended': 'VBD',
 'last': 'JJ',
 'month': 'NN',
 'by': 'IN',
 'a': 'SYM',
 'commission': 'NN',
 'hearing': 'NN',
 'officer': 'NN',
 'and': 'NNP',
 'barely': 'RB',
 'half': 'NN',
 'the': 'DT',
 'rise': 'NN',
 'sought': 'VBN',
 'utility': 'NN',
 '.': '.',
 'dummy': 'NN',
 'ruling': 'NN',
 'follows': 'VBZ',
 'host': 'NN',
 'of': 'IN',
 'problems': 'NNS',
 'including': 'IN',
 'major': 'JJ',
 'write-downs': 'NNS',
 '60': 'CD',
 'slash': 'VB',
 'in': 'IN',
 'common': 'JJ',
 'stock': 'NN',
 'dividend': 'NN',
 'departure': 'NN',
 'former': 'JJ',
 'Chairman': 'NNP',
 'Einar': 'dummy',
 'Greve': 'dummy',
 'during': 'IN',
 'company': 'NN',
 'investigation':

In [30]:
gd_output_df = pd.DataFrame(list(gd_output.items()), columns=['Words', 'POS Tags'])
gd_output_df = gd_output_df.drop(0)
gd_output_df

,Words,POS Tags
1,The,DT
2,Arizona,NNP
3,Corporations,NNS
4,Commission,NNP
5,authorized,VBD
...,...,...
15077,vowing,dummy
15078,rejects,VBZ
15079,adversarial,dummy
15080,hardball,dummy


In [31]:
save_as = "greedy.out"
gd_output_df.to_csv(save_as, header=None, sep='\t')

In [175]:
def update_df_columns(df: pd.DataFrame, new_columns_name: list, about: str) -> pd.DataFrame:  
    """Update the columns of the dataframe if first column is data needed"""  




    print("Update complete\n")    
    return updated_gd_output_df

# 4. Viterbi Decoding with HMM

1. [ ] Implement the viterbi decoding algorithm
2. [ ] Evaluate it on the development data
3. [ ] Predict the POS Tags of the sentences in the test data
4. [ ] Output the predictions in a file named `viterbi.out`, in the same format of training data
5. [ ] Question
    1. [ ] What is the accuracy on the dev data?

In [ ]:
# For s1, get key of largest prob outcome
np.argmax(np.array(list(t_probs.values())))

In [ ]:
updated_s1 = np.array(list(t_probs.keys()))[885]
updated_s1[1]

In [ ]:
updated_dev_df.head(40)

In [42]:
def viterbi_decoding(dev_df: pd.DataFrame, t_probs: dict, e_probs: dict, N_pos_tags: np.array):
    """Implement greedy decoding on the development file (words only) using the transition probability and emission probability. 
    
    
    ??? Furthermore, don't use POS Tag of development file, thus only use POS Tag from training data.

    Parameters
    ----------        
    dev_df: `pd.DataFrame`
        Dev file

    t_probs: `py dict`
        Tranision probabilities for POS Tag given previous POS Tag

    e_probs: `py dict`
        Emission probabilities for Word given POS Tags

    N_pos_tags: `np.array`
        All POS Tags found in the training file
    
    Return
    ------
    
    """

    """
    Base Cases:
    ----------
    v_pi: `py dictionary`
        Dictionary to store the length of each sentence (thus will have to reset to 0 every new sentence) and 
        all possible POS Tags (which will remain the same for each sentence). 
        
    v_pi[len_of_each_sentence, all_possible_tags] = t(all_pos_tags_in_dev_file | s) * e(words | all_pos_tags_in_dev_file)
    
    v_pi[0, all_pos_tags_in_dev_file] = t(t_find_pos_tag | t_given_pos_tag) * e(e_word | e_given_pos_tag)
    
    v_pi[0, dummy] = t(dummy | dummy) * e(The | dummy) = s1
    v_pi[0, DT] = t(DT | _) * e(The | DT) = s2
    v_pi[0, NNP] = t(NNP | _) * e(The | NNP) = s3

    v_pi_key: `py tuple`
        Tuple to store all possible combinations of keys for v_pi. 

    """

    previous_pos_tag = "dummy"
    v_pi = {}
    all_words_with_pos_tag = {}

    # start_algo_idx = 0

    """Base cases"""
    for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        # print("index", index, "with word", row['Word'], "and POS tag from dev is '", row['POS Tag'], "'2nd index", row["Index"])
        if row["Index"] == "dummy":
            break
        
        current_pos_tag = row['POS Tag']
        v_pi_key = (0, current_pos_tag)

        
        """Transition"""
        t_find_pos_tag = current_pos_tag
        t_given_pos_tag = previous_pos_tag
        # print(f"--- t({t_find_pos_tag} | {t_given_pos_tag})")

        """Emission"""
        e_word = row['Word']
        e_given_pos_tag = current_pos_tag
        # print(f"--- e({e_given_pos_tag} | {e_word})") # order this way to match e_probs dictionary

        """Transition * Emission"""
        t_key = (t_find_pos_tag, t_given_pos_tag)
        e_key = (e_given_pos_tag, e_word)
        # print(t_key in t_probs, e_key in e_probs)
        
        # IF-ELSE bc not all pairs will be found. If pair is found, use score, otherwise (pair isn't found) set score to 0.0.
        if t_key in t_probs and e_key in e_probs:
            if v_pi_key not in v_pi:
                t = t_probs[t_key]
                e = e_probs[e_key]
                score = t * e
                # print(f"--- FOUND 1x: pi{v_pi_key} = t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                v_pi[v_pi_key] = score
            else: # if key in v_pi dict found again, add scores to strenghten key
                # print(f"--- FOUND AGAIN: pi{v_pi_key} = t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                v_pi[v_pi_key] += score
            
        else:
            t = 0.0
            e = 0.0
            score = t * e
            # print(f"--- NOT FOUND: pi{v_pi_key} = t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
            v_pi[v_pi_key] = score
                        
        # print()
        all_words_with_pos_tag[row['Word']] = current_pos_tag
    # print("Fillings of 0th index for v_pi --- ", v_pi)

    # Fill v_pi with all possible options
    for i, row in dev_df.iterrows():
        for pos_tag in N_pos_tags:
            v_pi_key = (i + 1, pos_tag)
            v_pi[v_pi_key] = 0.0
        
    # print("Fillings of remaining pairs for v_pi --- ", v_pi)
            
    """Algo"""
    s_1 = np.argmax(np.array(list(v_pi.values())))
    updated_s1 = np.array(list(v_pi.keys()))[s_1]
    max_previous_pos_tag = updated_s1[1]

    # print()
    # print(f"Base case: {v_pi}")
    # print(f"s_1 (from the base case) index is {s_1} and the key at this index is {max_previous_pos_tag}")
    # print()

    track_pi_idx = 1 # TODO: Figure out when to reset. I think go until end of/ len of previous sentence
    
    for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        # print("index", index, "with word", row['Word'], "and POS tag from dev", row['POS Tag'], row["Index"])
        
        idx_j = track_pi_idx - 1
        # print("j - 1 = ", idx_j)
        previous_v_pi_key = (idx_j, max_previous_pos_tag)
        
        if index > 38:
            if row["Index"] == "dummy":
                break
            # pass
            if row["Index"] >= 1.0:
                current_pos_tag = row['POS Tag']
                # v_pi_key = (1, current_pos_tag)
                
                v_pi_key = (track_pi_idx, current_pos_tag)
                
                # print(f"Key of v_i is {v_pi_key} bc we  j is {track_pi_idx} and j - 1 is {idx_j}")
                

                """DP Algo
                pi[number_of_words_in_sentence * all_possible_tags] = t(all_pos_tags_in_dev_file | s) * e(words | all_pos_tags_in_dev_file)
                
                pi[1, all_pos_tags_in_dev_file] = t(t_find_pos_tag | t_given_pos_tag) * e(e_word | e_given_pos_tag)

                pi[1, DT] = max(pi[1, s1] * t(DT | s1) * e(The | DT) = s1
                (1, 'DT') = max(pi[1, s1] * t(DT | CD) * e(The | DT) 

                
                pi[2, NNP] = max(pi[2, s2] * t(NNP | s2) * e(Arizona | NNP) = s2
                pi[3, NNP] = max(pi[3, s3] * t(NNP | s3) * e(Corps... | NNP) = s3
                pi[4, NNP] = max(pi[4, s4] * t(NNP | s4) * e(Commission | NNP) = s4
            
                """
        
                """Transition"""
                t_find_pos_tag = current_pos_tag
                t_given_pos_tag = max_previous_pos_tag
                # print(f"--- t({t_find_pos_tag} | {t_given_pos_tag})")
        
                """Emission"""
                e_word = row['Word']
                e_given_pos_tag = current_pos_tag
                # print(f"--- e({e_given_pos_tag} | {e_word})") # order this way to match e_probs dictionary
        
                """Transition * Emission"""
                t_key = (t_find_pos_tag, t_given_pos_tag)
                e_key = (e_given_pos_tag, e_word)
                # print(t_key in t_probs, e_key in e_probs)
                # print("---previous_v_pi_key:", previous_v_pi_key, "with max_previous_pos_tag as", max_previous_pos_tag)
                
                # IF-ELSE bc not all pairs will be found. If pair is found, use score, otherwise (pair isn't found) set score to 0.0.
                if t_key in t_probs and e_key in e_probs and previous_v_pi_key in v_pi:
                    t = t_probs[t_key]
                    e = e_probs[e_key]
                    score = v_pi[previous_v_pi_key] * t * e
                    # v_pi[v_pi_key] = v_pi[previous_v_pi_key] * t * e
                    # print(f"--- pi{previous_v_pi_key} is FOUND in v_pi. Now, let's update v_pi at pi{v_pi_key}")
                    # print(f"--- pi{v_pi_key} = pi({previous_v_pi_key}) * t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                    # previous_v_pi_key = v_pi_key
             
                else:
                    t = 0.0
                    e = 0.0
                    # v_pi[v_pi_key] = 0.0 * t * e
                    score = 0.0 * t * e
                    # print(f"--- pi{previous_v_pi_key} is NOT FOUND in v_pi. Now, let's update v_pi at pi{v_pi_key}")
                    # print(f"--- pi{v_pi_key} = pi({previous_v_pi_key}) * t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                    # previous_v_pi_key = v_pi_key
                    
                                
                track_pi_idx += 1
                v_pi[previous_v_pi_key] = score
                
                
        
                # print("UPDATE")
                s_i = np.argmax(np.array(list(v_pi.values())))
                updated_s_i = np.array(list(v_pi.keys()))[s_i]
                # print(f"max value is at index: {s_i} in {v_pi} has key of {updated_s_i}")
                max_previous_pos_tag = updated_s_i[1]
                all_words_with_pos_tag[row['Word']] = max_previous_pos_tag
                # print("max_previous_pos_tag is", max_previous_pos_tag)
            # print(f"is {previous_v_pi_key} in v_pi")
            # print()

    return all_words_with_pos_tag    

In [43]:
# updated_train_df.head(25)

In [44]:
vd_output = viterbi_decoding(updated_dev_df, t_probs, e_probs, all_pos_tags)

  0%|                                                                                                                                                        | 45/137295 [00:27<23:31:13,  1.62it/s]


KeyboardInterrupt: 

Code below from --- https://en.wikipedia.org/wiki/Viterbi_algorithm

In [201]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        if st in emit_p and obs[0] in emit_p[st]:
            V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
        else:
            pass  # Placeholder, you may want to handle this case differently
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1] [states[0]] ["prob"] * trans_p[states[0]] [st] * emit_p[st] [obs[t]]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1] [prev_st] ["prob"] * trans_p[prev_st] [st] * emit_p[st] [obs[t]]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob
            V[t] [st] = {"prob": max_prob, "prev": prev_st_selected}

    for line in dptable(V):
        print(line)

    opt = []
    max_prob = 0.0
    best_st = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1] [previous] ["prev"])
        previous = V[t + 1] [previous] ["prev"]

    print ("The steps of states are " + " ".join(opt) + " with highest probability of %s" % max_prob)

def dptable(V):
    # Print a table of steps from dictionary
    yield " " * 5 + "     ".join(("%3d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%lf" % v[state] ["prob"]) for v in V)

In [181]:
obs = ("normal", "cold", "dizzy")
states = ("Healthy", "Fever")
start_p = {"Healthy": 0.6, "Fever": 0.4}
trans_p = {
    "Healthy": {"Healthy": 0.7, "Fever": 0.3},
    "Fever": {"Healthy": 0.4, "Fever": 0.6},
}
emit_p = {
    "Healthy": {"normal": 0.5, "cold": 0.4, "dizzy": 0.1},
    "Fever": {"normal": 0.1, "cold": 0.3, "dizzy": 0.6},
}

In [182]:
viterbi(obs,
        states,
        start_p,
        trans_p,
        emit_p)

       0       1       2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
The steps of states are Healthy Healthy Fever with highest probability of 0.01512


In [107]:
list(t_probs.items())[:7]

[(('dummy', 'NNP'), 0.19789104610393007),
 (('NNP', 'NNP'), 0.3782645420509543),
 (('NNP', ','), 0.13846908958086018),
 ((',', 'CD'), 0.021234939759036144),
 (('CD', 'NNS'), 0.15775891730703062),
 (('NNS', 'JJ'), 0.017196978862406887),
 (('JJ', ','), 0.029129343105320303)]

In [108]:
list(e_probs.items())[:7]

[(('dummy', ' '), 2.6165681092678842e-05),
 (('NNP', 'Pierre'), 6.84868961738654e-05),
 (('NNP', 'Vinken'), 2.2828965391288468e-05),
 ((',', ','), 0.9999139414802065),
 (('CD', '61'), 0.0007168253240050465),
 (('NNS', 'years'), 0.019530237301024905),
 (('JJ', 'old'), 0.003613599348534202)]

In [72]:
all_pos_tags

array(['dummy', 'NNP', ',', 'CD', 'NNS', 'JJ', 'MD', 'VB', 'DT', 'NN',
       'IN', '.', 'VBZ', 'VBG', 'CC', 'VBD', 'VBN', 'RB', 'TO', 'PRP',
       'RBR', 'WDT', 'VBP', 'RP', 'PRP$', 'JJS', 'POS', '``', 'EX', "''",
       'WP', ':', 'JJR', 'WRB', '$', 'NNPS', 'WP$', '-LRB-', '-RRB-',
       'PDT', 'RBS', 'FW', 'UH', 'SYM', 'LS', '#'], dtype=object)

In [119]:
def reformat_dictionary(original_dict):
    reformatted_dict = {}
    
    # Iterate through each key-value pair in the original dictionary
    for key, value in original_dict.items():
        # Extract the first string in the tuple as the outer key
        outer_key = key[0]
        
        # Extract the second string in the tuple as the inner key
        inner_key = key[1]
        
        # Check if the outer key already exists in the reformatted dictionary
        if outer_key in reformatted_dict:
            # If the outer key exists, append the inner key-value pair to its value
            reformatted_dict[outer_key][inner_key] = value
        else:
            # If the outer key doesn't exist, create a new inner dictionary
            reformatted_dict[outer_key] = {inner_key: value}
    
    return reformatted_dict

{'key1': {'value1': 'data1', 'value2': 'data2'}, 'key2': {'value3': 'data3'}}


In [123]:
# original_dict = {('key1', 'value1'): 'data1', ('key1', 'value2'): 'data2', ('key2', 'value3'): 'data3'}
t_prob_reformatted_dict = reformat_dictionary(t_probs)
e_prob_reformatted_dict = reformat_dictionary(e_probs)
# e_prob_reformatted_dict

In [206]:
all_pos_tags_reformatted = [elem for elem in all_pos_tags]

all_pos_tags_reformatted = tuple(all_pos_tags_reformatted)[1:]
all_pos_tags_reformatted

('NNP',
 ',',
 'CD',
 'NNS',
 'JJ',
 'MD',
 'VB',
 'DT',
 'NN',
 'IN',
 '.',
 'VBZ',
 'VBG',
 'CC',
 'VBD',
 'VBN',
 'RB',
 'TO',
 'PRP',
 'RBR',
 'WDT',
 'VBP',
 'RP',
 'PRP$',
 'JJS',
 'POS',
 '``',
 'EX',
 "''",
 'WP',
 ':',
 'JJR',
 'WRB',
 '$',
 'NNPS',
 'WP$',
 '-LRB-',
 '-RRB-',
 'PDT',
 'RBS',
 'FW',
 'UH',
 'SYM',
 'LS',
 '#')

In [203]:
words_in_df = updated_dev_df["Word"].values
# words_in_df = words_in_df[2:]
words_in_df

array([' ', 'The', 'Arizona', ..., 'winning', 'bidder', '.'], dtype=object)

In [204]:
all_words_reformatted = [elem for elem in words_in_df]
all_words_reformatted = tuple(all_words_reformatted)
# all_words_reformatted

In [162]:
train_pos_df = updated_train_df.copy()
true_false_series = train_pos_df['POS Tag'].value_counts()
updated_train_pos_df = pd.DataFrame(true_false_series)
updated_train_pos_df.reset_index(inplace = True)
# updated_train_pos_df

In [163]:

# Assuming you have a DataFrame named 'word_counts' with columns 'word' and 'count'
# Example:
# word_counts = pd.DataFrame({'word': ['0', 'the', 'dummy', '.', 'of'],
#                             'count': [46476, 39533, 38234, 37452, 22104]})

# Calculate the total count of all words
total_count = updated_train_pos_df['count'].sum()

# Calculate the probability of choosing each word
# updated_train_pos_df['probability'] = updated_train_pos_df['count'] / total_count
pos_probabilities = {row['POS Tag']: row['count'] / total_count for _, row in updated_train_pos_df.iterrows()}


# Display the probabilities
# for get_pos_tag, probability in pos_probabilities.items():
#     print(f"{pos_probabilities}, '{get_pos_tag}': {probability:.4f}")

In [178]:
# e_prob_reformatted_dict

In [209]:
obs = all_words_reformatted
states = all_pos_tags_reformatted
start_p = pos_probabilities
trans_p = t_prob_reformatted_dict
emit_p = e_prob_reformatted_dict

In [210]:
viterbi(obs,
        states,
        start_p,
        trans_p,
        emit_p)

KeyError: 'NNP'

In [ ]:
# obs = ("normal", "cold", "dizzy")
# states = ("Healthy", "Fever")
# start_p = {"Healthy": 0.6, "Fever": 0.4}
# trans_p = {
#     "Healthy": {"Healthy": 0.7, "Fever": 0.3},
#     "Fever": {"Healthy": 0.4, "Fever": 0.6},
# }
# emit_p = {
#     "Healthy": {"normal": 0.5, "cold": 0.4, "dizzy": 0.1},
#     "Fever": {"normal": 0.1, "cold": 0.3, "dizzy": 0.6},
# }

In [187]:
t_prob_reformatted_dict

{'dummy': {'NNP': 0.19789104610393007,
  'DT': 0.21911141347009264,
  'IN': 0.1288398137003506,
  'PRP': 0.06148935056779528,
  'EX': 0.004238840337013972,
  '``': 0.07472918520069077,
  'CD': 0.011225077188759224,
  'RBR': 0.0020932544874143074,
  'NNS': 0.041237113402061855,
  'NN': 0.0411847820398765,
  'JJ': 0.041708095661730074,
  'JJR': 0.0017007692710241248,
  'RB': 0.05604688890051808,
  'WRB': 0.00609660369459417,
  'CC': 0.05691035637657648,
  'VBG': 0.012010047621539588,
  'WDT': 0.0008111361138730441,
  'VBN': 0.005834946883667382,
  '-LRB-': 0.003427704223140928,
  'VB': 0.0030613846878434245,
  'WP': 0.003113716050028782,
  'PRP$': 0.007797372965618295,
  'TO': 0.0035323669475116437,
  'JJS': 0.00248573970380449,
  'NNPS': 0.0020409231252289496,
  'VBZ': 0.001517609503375373,
  'VBD': 0.0007588047516876865,
  'LS': 0.0009157988382437595,
  "''": 0.0003663195352975038,
  ':': 0.002799727876916636,
  'VBP': 0.0003663195352975038,
  'PDT': 0.0007326390705950076,
  'UH': 0.00